In [2]:
#pip install spacy 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.1/493.1 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.3/880.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import spacy
import numpy as np
from sklearn.manifold import MDS
import pandas as pd

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Path to the dataset folder
dataset_folder = 'dataset'

# List to store text data from all files
all_text_data = []

# Entity types of interest
entity_types = ['PERSON', 'GPE', 'DATE', 'ORG', 'CARDINAL']

# Loop through files in the dataset folder
for filename in os.listdir(dataset_folder):
    # Read the contents of the file
    with open(os.path.join(dataset_folder, filename), 'r', encoding='utf-8') as file:
        text_data = file.read()
        all_text_data.append((filename, text_data))  # Store filename along with text

# Initialize an empty DataFrame to store entity counts per document
entity_matrix = pd.DataFrame(0, index=[fname for fname, _ in all_text_data], columns=entity_types)

# Populate the DataFrame with entity counts
for filename, text in all_text_data:
    # Process the text using spaCy
    doc = nlp(text)
    # Count the entities of interest
    entity_counts = {etype: 0 for etype in entity_types}
    for ent in doc.ents:
        if ent.label_ in entity_types:
            entity_counts[ent.label_] += 1
    # Update the DataFrame
    entity_matrix.loc[filename] = [entity_counts[etype] for etype in entity_types]

# Apply Multidimensional Scaling
mds = MDS(n_components=2, random_state=42)
mds_coords = mds.fit_transform(entity_matrix.values)

# Create a DataFrame for the MDS results
mds_results = pd.DataFrame(mds_coords, index=entity_matrix.index, columns=['MDS1', 'MDS2'])
print(mds_results)

# Optionally, save the MDS results to a CSV file
output_csv = "mds_results.csv"
mds_results.to_csv(output_csv)
print(f"MDS results saved to {output_csv}")


                      MDS1       MDS2
CIA_01            9.850102  -0.667158
CIA_02            6.570594   4.167886
CIA_03            7.158469  -1.316559
CIA_04            9.430058  -0.146963
CIA_05           12.158756  -0.418011
...                    ...        ...
NSA_22            8.143254  -1.916215
NSA_docs.txt   -109.672427 -54.611715
USCBP_01          9.556821   6.860964
USCBP_02         11.384500   1.291512
USCBP_docs.txt    1.205559   8.379009

[116 rows x 2 columns]
MDS results saved to mds_results.csv


In [3]:
import pandas as pd

# Generate JSON file with index
mds_results.reset_index().to_json('mds_data.json', orient='records')